In [1]:
import torch
import numpy as np

In [2]:
torch.__version__

'1.13.1'

In [3]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.io import wavfile
from scipy.signal import fftconvolve
import IPython
import pyroomacoustics as pra

from pyroomacoustics.directivities import (
    DirectivityPattern,
    DirectionVector,
    CardioidFamily,
)

In [4]:
def room_properties(absorption, size):
    if absorption=='high':
        absor = 0.7
    elif absorption=='medium':
        absor = 0.3
    elif absorption=='low':
        absor = 0.1
    else:
        raise ValueError("The absorption parameter can only take values ['low', 'medium', 'high']")
    
    if size=='large':
        size_coef = 5.0
    elif size=='medium':
        size_coef = 2.5
    elif size=='small':
        size_coef = 1.0
    else:
        raise ValueError("The absorption parameter can only take values ['low', 'medium', 'high']")
    return absor, size_coef

In [5]:
def clip(signal, high, low):
    """Clip a signal from above at high and from below at low."""
    s = signal.copy()

    s[np.where(s > high)] = high
    s[np.where(s < low)] = low

    return s

def normalize(signal, bits=None):
    """
    normalize to be in a given range. The default is to normalize the maximum
    amplitude to be one. An optional argument allows to normalize the signal
    to be within the range of a given signed integer representation of bits.
    """

    s = signal.copy()

    s /= np.abs(s).max()

    # if one wants to scale for bits allocated
    if bits is not None:
        s *= 2 ** (bits - 1) - 1
        s = clip(s, 2 ** (bits - 1) - 1, -(2 ** (bits - 1)))

    return s
    
def to_wav(filename, fs, signals, norm=False, bitdepth=np.float):
    """
    Save all the signals to wav files.

    Parameters
    ----------
    filename: str
        the name of the file
    mono: bool, optional
        if true, records only the center channel floor(M / 2) (default
        `False`)
    norm: bool, optional
        if true, normalize the signal to fit in the dynamic range (default
        `False`)
    bitdepth: int, optional
        the format of output samples [np.int8/16/32/64 or np.float
        (default)]
    """
    from scipy.io import wavfile

    signal = signals.T  # each column is a channel

    float_types = [float, np.float, np.float32, np.float64]

    if bitdepth in float_types:
        bits = None
    elif bitdepth is np.int8:
        bits = 8
    elif bitdepth is np.int16:
        bits = 16
    elif bitdepth is np.int32:
        bits = 32
    elif bitdepth is np.int64:
        bits = 64
    else:
        raise NameError("No such type.")

    if norm:
        signal = normalize(signal, bits=bits)

    signal = np.array(signal, dtype=bitdepth)

    wavfile.write(filename, fs, signal)

/tmp/ipykernel_43156/3403348603.py:28: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  def to_wav(filename, fs, signals, norm=False, bitdepth=np.float):


# G1

In [6]:
import glob

In [7]:
a = glob.glob('/nvme/zhiyong/voxceleb/vox1/dev/wav/*')

In [8]:
b = a[0:100]

In [9]:
b_list = []
for i in b:
    b_list.extend(glob.glob(i+'/*'))

In [10]:
b_list

['/nvme/zhiyong/voxceleb/vox1/dev/wav/id10143/ioty72DM3MQ',
 '/nvme/zhiyong/voxceleb/vox1/dev/wav/id10143/crXv8XQjLLE',
 '/nvme/zhiyong/voxceleb/vox1/dev/wav/id10143/69XxH60bd4Y',
 '/nvme/zhiyong/voxceleb/vox1/dev/wav/id10143/RsdRVYywZ74',
 '/nvme/zhiyong/voxceleb/vox1/dev/wav/id10143/QEFFrsJSASk',
 '/nvme/zhiyong/voxceleb/vox1/dev/wav/id10143/34YA89TlYlI',
 '/nvme/zhiyong/voxceleb/vox1/dev/wav/id10143/p1RUz7Gkb-I',
 '/nvme/zhiyong/voxceleb/vox1/dev/wav/id10143/SaEbiRYY_is',
 '/nvme/zhiyong/voxceleb/vox1/dev/wav/id10143/6aUi7zTDQ6g',
 '/nvme/zhiyong/voxceleb/vox1/dev/wav/id10143/nHHR2PDyr3Q',
 '/nvme/zhiyong/voxceleb/vox1/dev/wav/id10143/Ve-PgsDlTqQ',
 '/nvme/zhiyong/voxceleb/vox1/dev/wav/id10143/eve4ji2uJo8',
 '/nvme/zhiyong/voxceleb/vox1/dev/wav/id10143/2LSiFyZN5-Y',
 '/nvme/zhiyong/voxceleb/vox1/dev/wav/id10143/8df9woqxqXE',
 '/nvme/zhiyong/voxceleb/vox1/dev/wav/id10122/5lzLbwgdGTM',
 '/nvme/zhiyong/voxceleb/vox1/dev/wav/id10122/WQBa3omfC9U',
 '/nvme/zhiyong/voxceleb/vox1/dev/wav/id

In [11]:
def generate_wave(path, out_path):
    absor, size_coef = room_properties('high', 'small')
    room = pra.ShoeBox(
        p=size_coef*np.array([4, 3, 2.5]),
        materials=pra.Material(absor),
        fs=16000,
        max_order=3,
        ray_tracing=True,
        air_absorption=True,
    )
    room.set_ray_tracing(receiver_radius=0.1, n_rays=10000, energy_thres=1e-5)

    # specify signal source
    fs, signal = wavfile.read(path)

    # add source to 2D room
    room.add_source([1.,1.,1.], signal=signal)

    # add microphone array
    room.add_microphone(loc=[3.5,2.5,1.])

    room.simulate()
    
    to_wav(out_path, fs, room.mic_array.signals[0,:], norm=True, bitdepth=np.int16)

In [12]:
glob.glob('/nvme/zhiyong/voxceleb/vox1/dev/wav/id10143/ioty72DM3MQ/*')

['/nvme/zhiyong/voxceleb/vox1/dev/wav/id10143/ioty72DM3MQ/00004.wav',
 '/nvme/zhiyong/voxceleb/vox1/dev/wav/id10143/ioty72DM3MQ/00001.wav',
 '/nvme/zhiyong/voxceleb/vox1/dev/wav/id10143/ioty72DM3MQ/00006.wav',
 '/nvme/zhiyong/voxceleb/vox1/dev/wav/id10143/ioty72DM3MQ/00005.wav',
 '/nvme/zhiyong/voxceleb/vox1/dev/wav/id10143/ioty72DM3MQ/00003.wav',
 '/nvme/zhiyong/voxceleb/vox1/dev/wav/id10143/ioty72DM3MQ/00002.wav']

In [13]:
for i in b_list:
    c = glob.glob(i+'/*')
    for j in c:
        label = j.split('/')[-3]
        out_path = '/nvme/zhiyong/fllearn_data/G1/'
        generate_wave(j, out_path+label+'_'+j.split('/')[-2]+'_'+j.split('/')[-1])
        with open(out_path+'train_list_G1.txt', 'a') as f:
            f.write(label+' '+out_path+label+'_'+j.split('/')[-2]+'_'+j.split('/')[-1]+'\n')

/tmp/ipykernel_43156/3403348603.py:50: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  float_types = [float, np.float, np.float32, np.float64]


# G2

In [14]:
import glob

In [15]:
a = glob.glob('/nvme/zhiyong/voxceleb/vox1/dev/wav/*')

In [16]:
b = a[100:200]

In [17]:
b_list = []
for i in b:
    b_list.extend(glob.glob(i+'/*'))

In [18]:
b_list

['/nvme/zhiyong/voxceleb/vox1/dev/wav/id10381/wCP1w4zr1hY',
 '/nvme/zhiyong/voxceleb/vox1/dev/wav/id10381/OQkL_jLpx3A',
 '/nvme/zhiyong/voxceleb/vox1/dev/wav/id10381/RL-O0Ut_7Ds',
 '/nvme/zhiyong/voxceleb/vox1/dev/wav/id10381/uwTOyHjDjW8',
 '/nvme/zhiyong/voxceleb/vox1/dev/wav/id10381/nK1lfRZQWwk',
 '/nvme/zhiyong/voxceleb/vox1/dev/wav/id10381/EPyAUoJWqig',
 '/nvme/zhiyong/voxceleb/vox1/dev/wav/id10381/E3ohJwRKe9E',
 '/nvme/zhiyong/voxceleb/vox1/dev/wav/id10381/1O5YJIGl1X4',
 '/nvme/zhiyong/voxceleb/vox1/dev/wav/id10381/0pc7P1-EZ5o',
 '/nvme/zhiyong/voxceleb/vox1/dev/wav/id10381/3K_Wgn7jj8k',
 '/nvme/zhiyong/voxceleb/vox1/dev/wav/id10381/2BZvH0eCUHw',
 '/nvme/zhiyong/voxceleb/vox1/dev/wav/id10381/cOWp-pyuFWA',
 '/nvme/zhiyong/voxceleb/vox1/dev/wav/id10381/WtybwhFuhoQ',
 '/nvme/zhiyong/voxceleb/vox1/dev/wav/id10381/USQqk2U03hI',
 '/nvme/zhiyong/voxceleb/vox1/dev/wav/id10381/TC9hxG_pVzM',
 '/nvme/zhiyong/voxceleb/vox1/dev/wav/id10381/RywJYlB8Hjs',
 '/nvme/zhiyong/voxceleb/vox1/dev/wav/id

In [19]:
def generate_wave(path, out_path):
    absor, size_coef = room_properties('low', 'large')
    room = pra.ShoeBox(
        p=np.array([4*size_coef, 3*size_coef, 2.5]),
        materials=pra.Material(absor),
        fs=16000,
        max_order=3,
        ray_tracing=True,
        air_absorption=True,
    )
    room.set_ray_tracing(receiver_radius=0.1, n_rays=10000, energy_thres=1e-5)

    # specify signal source
    fs, signal = wavfile.read(path)

    # add source to 2D room
    room.add_source([1.,1.,1.], signal=signal)

    # add microphone array
    room.add_microphone(loc=[17.,12.,1.])

    room.simulate()
    to_wav(out_path, fs, room.mic_array.signals[0,:], norm=True, bitdepth=np.int16)

In [20]:
glob.glob('/nvme/zhiyong/voxceleb/vox1/dev/wav/id10143/ioty72DM3MQ/*')

['/nvme/zhiyong/voxceleb/vox1/dev/wav/id10143/ioty72DM3MQ/00004.wav',
 '/nvme/zhiyong/voxceleb/vox1/dev/wav/id10143/ioty72DM3MQ/00001.wav',
 '/nvme/zhiyong/voxceleb/vox1/dev/wav/id10143/ioty72DM3MQ/00006.wav',
 '/nvme/zhiyong/voxceleb/vox1/dev/wav/id10143/ioty72DM3MQ/00005.wav',
 '/nvme/zhiyong/voxceleb/vox1/dev/wav/id10143/ioty72DM3MQ/00003.wav',
 '/nvme/zhiyong/voxceleb/vox1/dev/wav/id10143/ioty72DM3MQ/00002.wav']

In [21]:
for i in b_list:
    c = glob.glob(i+'/*')
    for j in c:
        label = j.split('/')[-3]
        out_path = '/nvme/zhiyong/fllearn_data/G2/'
        generate_wave(j, out_path+label+'_'+j.split('/')[-2]+'_'+j.split('/')[-1])
        with open(out_path+'train_list_G2.txt', 'a') as f:
            f.write(label+' '+out_path+label+'_'+j.split('/')[-2]+'_'+j.split('/')[-1]+'\n')

/tmp/ipykernel_43156/3403348603.py:50: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  float_types = [float, np.float, np.float32, np.float64]


# G3

In [22]:
import glob

In [23]:
a = glob.glob('/nvme/zhiyong/voxceleb/vox1/dev/wav/*')

In [24]:
b = a[200:300]

In [25]:
b_list = []
for i in b:
    b_list.extend(glob.glob(i+'/*'))

In [26]:
b_list

['/nvme/zhiyong/voxceleb/vox1/dev/wav/id10036/qEpBKVTog9M',
 '/nvme/zhiyong/voxceleb/vox1/dev/wav/id10036/XBnhi1cQtX4',
 '/nvme/zhiyong/voxceleb/vox1/dev/wav/id10036/VOtL34w9e7Y',
 '/nvme/zhiyong/voxceleb/vox1/dev/wav/id10036/eN_SgDPbwRA',
 '/nvme/zhiyong/voxceleb/vox1/dev/wav/id10036/qARhKvGWef0',
 '/nvme/zhiyong/voxceleb/vox1/dev/wav/id10036/ywS-iusuJJo',
 '/nvme/zhiyong/voxceleb/vox1/dev/wav/id10036/0GgHhOqUrUw',
 '/nvme/zhiyong/voxceleb/vox1/dev/wav/id10036/EAsTZq6lLHI',
 '/nvme/zhiyong/voxceleb/vox1/dev/wav/id10036/NKuVgEn0sr8',
 '/nvme/zhiyong/voxceleb/vox1/dev/wav/id10036/CzoVmf9MMeE',
 '/nvme/zhiyong/voxceleb/vox1/dev/wav/id10036/XkXudadCQHo',
 '/nvme/zhiyong/voxceleb/vox1/dev/wav/id10036/JD81EN98tI8',
 '/nvme/zhiyong/voxceleb/vox1/dev/wav/id10036/MeU1v-T0hLI',
 '/nvme/zhiyong/voxceleb/vox1/dev/wav/id10036/q6C7Uh1bzio',
 '/nvme/zhiyong/voxceleb/vox1/dev/wav/id10036/sjqz97b0XuI',
 '/nvme/zhiyong/voxceleb/vox1/dev/wav/id10036/ammRSKiXIZY',
 '/nvme/zhiyong/voxceleb/vox1/dev/wav/id

In [27]:
def generate_wave(path, out_path):
    absor, size_coef = room_properties('medium', 'medium')
    room = pra.ShoeBox(
        p=np.array([4*size_coef, 3*size_coef, 2.5]),
        materials=pra.Material(absor),
        fs=16000,
        max_order=3,
        ray_tracing=True,
        air_absorption=True,
    )
    room.set_ray_tracing(receiver_radius=0.1, n_rays=10000, energy_thres=1e-5)

    # specify signal source
    fs, signal = wavfile.read(path)

    # add source to 2D room
    room.add_source([1.,1.,1.], signal=signal)

    # add microphone array
    room.add_microphone(loc=[8.,6.,1.])

    room.simulate()
    to_wav(out_path, fs, room.mic_array.signals[0,:], norm=True, bitdepth=np.int16)

In [28]:
glob.glob('/nvme/zhiyong/voxceleb/vox1/dev/wav/id10143/ioty72DM3MQ/*')

['/nvme/zhiyong/voxceleb/vox1/dev/wav/id10143/ioty72DM3MQ/00004.wav',
 '/nvme/zhiyong/voxceleb/vox1/dev/wav/id10143/ioty72DM3MQ/00001.wav',
 '/nvme/zhiyong/voxceleb/vox1/dev/wav/id10143/ioty72DM3MQ/00006.wav',
 '/nvme/zhiyong/voxceleb/vox1/dev/wav/id10143/ioty72DM3MQ/00005.wav',
 '/nvme/zhiyong/voxceleb/vox1/dev/wav/id10143/ioty72DM3MQ/00003.wav',
 '/nvme/zhiyong/voxceleb/vox1/dev/wav/id10143/ioty72DM3MQ/00002.wav']

In [29]:
for i in b_list:
    c = glob.glob(i+'/*')
    for j in c:
        label = j.split('/')[-3]
        out_path = '/nvme/zhiyong/fllearn_data/G3/'
        generate_wave(j, out_path+label+'_'+j.split('/')[-2]+'_'+j.split('/')[-1])
        with open(out_path+'train_list_G3.txt', 'a') as f:
            f.write(label+' '+out_path+label+'_'+j.split('/')[-2]+'_'+j.split('/')[-1]+'\n')

/tmp/ipykernel_43156/3403348603.py:50: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  float_types = [float, np.float, np.float32, np.float64]


# G4

In [30]:
import glob

In [31]:
a = glob.glob('/nvme/zhiyong/voxceleb/vox1/dev/wav/*')

In [32]:
b = a[300:400]

In [33]:
b_list = []
for i in b:
    b_list.extend(glob.glob(i+'/*'))

In [34]:
b_list

['/nvme/zhiyong/voxceleb/vox1/dev/wav/id10514/Vdee6WzjFvw',
 '/nvme/zhiyong/voxceleb/vox1/dev/wav/id10514/3t8MrilBjns',
 '/nvme/zhiyong/voxceleb/vox1/dev/wav/id10514/9Ne7ROSlmbo',
 '/nvme/zhiyong/voxceleb/vox1/dev/wav/id10514/qeyQKRSbAlE',
 '/nvme/zhiyong/voxceleb/vox1/dev/wav/id10514/FKvHYfqQNwQ',
 '/nvme/zhiyong/voxceleb/vox1/dev/wav/id10514/9xBcUJ0Mt4w',
 '/nvme/zhiyong/voxceleb/vox1/dev/wav/id10514/5guvacjkUq4',
 '/nvme/zhiyong/voxceleb/vox1/dev/wav/id10514/T-xk-ZJheAU',
 '/nvme/zhiyong/voxceleb/vox1/dev/wav/id10514/xCRzr9FEoMQ',
 '/nvme/zhiyong/voxceleb/vox1/dev/wav/id10837/25CRXKcRFuk',
 '/nvme/zhiyong/voxceleb/vox1/dev/wav/id10837/gWlfi3ek7tk',
 '/nvme/zhiyong/voxceleb/vox1/dev/wav/id10837/P4-AZb5WuJ4',
 '/nvme/zhiyong/voxceleb/vox1/dev/wav/id10837/6_I23D4iiUU',
 '/nvme/zhiyong/voxceleb/vox1/dev/wav/id10837/CgnS14PmXHA',
 '/nvme/zhiyong/voxceleb/vox1/dev/wav/id10837/o9QzZwyCr5s',
 '/nvme/zhiyong/voxceleb/vox1/dev/wav/id10837/Rek5wuRFKCo',
 '/nvme/zhiyong/voxceleb/vox1/dev/wav/id

In [35]:
def generate_wave(path, out_path):
    absor, size_coef = room_properties('medium', 'medium')
    room = pra.ShoeBox(
        p=np.array([4*size_coef, 3*size_coef, 2.5]),
        materials=pra.Material(absor),
        fs=16000,
        max_order=3,
        ray_tracing=True,
        air_absorption=True,
    )
    room.set_ray_tracing(receiver_radius=0.1, n_rays=10000, energy_thres=1e-5)

    # specify signal source
    fs, signal = wavfile.read(path)

    # add source to 2D room
    room.add_source([1.,1.,1.], signal=signal)

    # add noise source
    fs, noise = wavfile.read("/nvme/zhiyong/musan/noise/free-sound/noise-free-sound-0000.wav")

    room.add_source([4.,7.,1.], signal=noise[:len(signal)], delay=0.)

    # add microphone array
    room.add_microphone(loc=[8.,4.,1.])

    room.simulate()
    to_wav(out_path, fs, room.mic_array.signals[0,:], norm=True, bitdepth=np.int16)

In [36]:
glob.glob('/nvme/zhiyong/voxceleb/vox1/dev/wav/id10143/ioty72DM3MQ/*')

['/nvme/zhiyong/voxceleb/vox1/dev/wav/id10143/ioty72DM3MQ/00004.wav',
 '/nvme/zhiyong/voxceleb/vox1/dev/wav/id10143/ioty72DM3MQ/00001.wav',
 '/nvme/zhiyong/voxceleb/vox1/dev/wav/id10143/ioty72DM3MQ/00006.wav',
 '/nvme/zhiyong/voxceleb/vox1/dev/wav/id10143/ioty72DM3MQ/00005.wav',
 '/nvme/zhiyong/voxceleb/vox1/dev/wav/id10143/ioty72DM3MQ/00003.wav',
 '/nvme/zhiyong/voxceleb/vox1/dev/wav/id10143/ioty72DM3MQ/00002.wav']

In [37]:
for i in b_list:
    c = glob.glob(i+'/*')
    for j in c:
        label = j.split('/')[-3]
        out_path = '/nvme/zhiyong/fllearn_data/G4/'
        generate_wave(j, out_path+label+'_'+j.split('/')[-2]+'_'+j.split('/')[-1])
        with open(out_path+'train_list_G4.txt', 'a') as f:
            f.write(label+' '+out_path+label+'_'+j.split('/')[-2]+'_'+j.split('/')[-1]+'\n')

/tmp/ipykernel_43156/3403348603.py:50: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  float_types = [float, np.float, np.float32, np.float64]


# G5

In [38]:
import glob

In [39]:
a = glob.glob('/nvme/zhiyong/voxceleb/vox1/dev/wav/*')

In [40]:
b = a[400:500]

In [41]:
b_list = []
for i in b:
    b_list.extend(glob.glob(i+'/*'))

In [42]:
b_list

['/nvme/zhiyong/voxceleb/vox1/dev/wav/id10475/cdafKPdx-1o',
 '/nvme/zhiyong/voxceleb/vox1/dev/wav/id10475/tAY8zGzb4gc',
 '/nvme/zhiyong/voxceleb/vox1/dev/wav/id10475/hDBMV_0Vz4E',
 '/nvme/zhiyong/voxceleb/vox1/dev/wav/id10475/Wm-ph0O3ZUM',
 '/nvme/zhiyong/voxceleb/vox1/dev/wav/id10475/jIqEzrGz7iM',
 '/nvme/zhiyong/voxceleb/vox1/dev/wav/id10475/Rt0Ww-WF7t4',
 '/nvme/zhiyong/voxceleb/vox1/dev/wav/id10475/EhYmFkR-PrE',
 '/nvme/zhiyong/voxceleb/vox1/dev/wav/id10475/HTL8iLI75TY',
 '/nvme/zhiyong/voxceleb/vox1/dev/wav/id10475/HyKhBpBZmcQ',
 '/nvme/zhiyong/voxceleb/vox1/dev/wav/id10475/TiHRF2zNMxo',
 '/nvme/zhiyong/voxceleb/vox1/dev/wav/id10475/tDB4enof1Xg',
 '/nvme/zhiyong/voxceleb/vox1/dev/wav/id10475/mjFneMRu_bQ',
 '/nvme/zhiyong/voxceleb/vox1/dev/wav/id10475/Y6YBfcuY_nc',
 '/nvme/zhiyong/voxceleb/vox1/dev/wav/id10475/jUSC4i_eGHs',
 '/nvme/zhiyong/voxceleb/vox1/dev/wav/id10475/Oyf8Fx_3xBs',
 '/nvme/zhiyong/voxceleb/vox1/dev/wav/id10475/rMPi4FHUuJY',
 '/nvme/zhiyong/voxceleb/vox1/dev/wav/id

In [43]:
def generate_wave(path, out_path):
    fft_len = 512
    Lg_t = 0.100                # filter size in seconds
    Lg = np.ceil(Lg_t*16000)       # in samples
    absor, size_coef = room_properties('medium', 'medium')
    room = pra.ShoeBox(
        p=np.array([4*size_coef, 3*size_coef, 2.5]),
        materials=pra.Material(absor),
        fs=16000,
        max_order=3,
        ray_tracing=True,
        air_absorption=True,
    )
    room.set_ray_tracing(receiver_radius=0.1, n_rays=10000, energy_thres=1e-5)

    # specify signal source
    fs, signal = wavfile.read(path)

    # add source to 2D room
    room.add_source([1.,1.,1.], signal=signal)

    # add microphone array
    R = pra.circular_2D_array(center=[8.,2.], M=6, phi0=0, radius=0.05)
    R = np.concatenate((R, np.ones((1, 6))))
    # dir_list = []
    # for i in range(6):
    #     dir_list.append(CardioidFamily(
    #     orientation=DirectionVector(azimuth=0, colatitude=90, degrees=True),
    #     pattern_enum=DirectivityPattern.OMNI,))

    # mics = pra.MicrophoneArray(R, room.fs, directivity=dir_list)
    mics = pra.Beamformer(R, fs, N=fft_len, Lg=Lg)
    room.add_microphone_array(mics)

    # Compute DAS weights
    mics.rake_delay_and_sum_weights(room.sources[0][:1])

    room.simulate()
    signal_das = mics.process(FD=False)
    to_wav(out_path, fs, signal_das, norm=True, bitdepth=np.int16)

In [44]:
glob.glob('/nvme/zhiyong/voxceleb/vox1/dev/wav/id10143/ioty72DM3MQ/*')

['/nvme/zhiyong/voxceleb/vox1/dev/wav/id10143/ioty72DM3MQ/00004.wav',
 '/nvme/zhiyong/voxceleb/vox1/dev/wav/id10143/ioty72DM3MQ/00001.wav',
 '/nvme/zhiyong/voxceleb/vox1/dev/wav/id10143/ioty72DM3MQ/00006.wav',
 '/nvme/zhiyong/voxceleb/vox1/dev/wav/id10143/ioty72DM3MQ/00005.wav',
 '/nvme/zhiyong/voxceleb/vox1/dev/wav/id10143/ioty72DM3MQ/00003.wav',
 '/nvme/zhiyong/voxceleb/vox1/dev/wav/id10143/ioty72DM3MQ/00002.wav']

In [45]:
for i in b_list:
    c = glob.glob(i+'/*')
    for j in c:
        label = j.split('/')[-3]
        out_path = '/nvme/zhiyong/fllearn_data/G5/'
        generate_wave(j, out_path+label+'_'+j.split('/')[-2]+'_'+j.split('/')[-1])
        with open(out_path+'train_list_G5.txt', 'a') as f:
            f.write(label+' '+out_path+label+'_'+j.split('/')[-2]+'_'+j.split('/')[-1]+'\n')

/tmp/ipykernel_43156/3403348603.py:50: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  float_types = [float, np.float, np.float32, np.float64]


# G6

In [46]:
import glob

In [47]:
a = glob.glob('/nvme/zhiyong/voxceleb/vox1/dev/wav/*')

In [48]:
b = a[500:600]

In [49]:
b_list = []
for i in b:
    b_list.extend(glob.glob(i+'/*'))

In [50]:
b_list

['/nvme/zhiyong/voxceleb/vox1/dev/wav/id10146/jkpEEFySgIY',
 '/nvme/zhiyong/voxceleb/vox1/dev/wav/id10146/XseYDfLdjmc',
 '/nvme/zhiyong/voxceleb/vox1/dev/wav/id10146/dGTzmTvWtzY',
 '/nvme/zhiyong/voxceleb/vox1/dev/wav/id10146/vVBxA3QHlW4',
 '/nvme/zhiyong/voxceleb/vox1/dev/wav/id10146/3GpKr8kPlsc',
 '/nvme/zhiyong/voxceleb/vox1/dev/wav/id10146/nM-OJzRHMfU',
 '/nvme/zhiyong/voxceleb/vox1/dev/wav/id10146/QfJ3Jqdio5o',
 '/nvme/zhiyong/voxceleb/vox1/dev/wav/id10146/cfRB26fxHaw',
 '/nvme/zhiyong/voxceleb/vox1/dev/wav/id10146/mxSucaiCHmY',
 '/nvme/zhiyong/voxceleb/vox1/dev/wav/id10146/zRXyp-yH3Uo',
 '/nvme/zhiyong/voxceleb/vox1/dev/wav/id10146/ZC-VOhJpIXU',
 '/nvme/zhiyong/voxceleb/vox1/dev/wav/id10146/6JT84QLtKsI',
 '/nvme/zhiyong/voxceleb/vox1/dev/wav/id10146/XzFXCNxJE-0',
 '/nvme/zhiyong/voxceleb/vox1/dev/wav/id11227/50Ra-5pIxIE',
 '/nvme/zhiyong/voxceleb/vox1/dev/wav/id11227/g4ObxCoFgGc',
 '/nvme/zhiyong/voxceleb/vox1/dev/wav/id11227/uitE9kM2GtU',
 '/nvme/zhiyong/voxceleb/vox1/dev/wav/id

In [51]:
def generate_wave(path, out_path):
    fft_len = 512
    Lg_t = 0.100                # filter size in seconds
    Lg = np.ceil(Lg_t*16000)       # in samples
    absor, size_coef = room_properties('medium', 'medium')
    room = pra.ShoeBox(
        p=np.array([4*size_coef, 3*size_coef, 2.5]),
        materials=pra.Material(absor),
        fs=16000,
        max_order=3,
        ray_tracing=True,
        air_absorption=True,
    )
    room.set_ray_tracing(receiver_radius=0.1, n_rays=10000, energy_thres=1e-5)

    # specify signal source
    fs, signal = wavfile.read(path)

    # add source to 2D room
    room.add_source([1.,1.,1.], signal=signal)

    # add noise source
    fs, noise = wavfile.read("/nvme/zhiyong/musan/noise/free-sound/noise-free-sound-0000.wav")

    room.add_source([8.,7.,1.], signal=noise[:len(signal)], delay=0.)

    # add microphone array
    R = pra.circular_2D_array(center=[8.,2.], M=6, phi0=0, radius=0.05)
    R = np.concatenate((R, np.ones((1, 6))))

    mics = pra.Beamformer(R, fs, N=fft_len, Lg=Lg)
    room.add_microphone_array(mics)

    # Compute DAS weights
    mics.rake_delay_and_sum_weights(room.sources[0][:1])

    room.simulate()
    signal_das = mics.process(FD=False)
    to_wav(out_path, fs, signal_das, norm=True, bitdepth=np.int16)

In [52]:
glob.glob('/nvme/zhiyong/voxceleb/vox1/dev/wav/id10143/ioty72DM3MQ/*')

['/nvme/zhiyong/voxceleb/vox1/dev/wav/id10143/ioty72DM3MQ/00004.wav',
 '/nvme/zhiyong/voxceleb/vox1/dev/wav/id10143/ioty72DM3MQ/00001.wav',
 '/nvme/zhiyong/voxceleb/vox1/dev/wav/id10143/ioty72DM3MQ/00006.wav',
 '/nvme/zhiyong/voxceleb/vox1/dev/wav/id10143/ioty72DM3MQ/00005.wav',
 '/nvme/zhiyong/voxceleb/vox1/dev/wav/id10143/ioty72DM3MQ/00003.wav',
 '/nvme/zhiyong/voxceleb/vox1/dev/wav/id10143/ioty72DM3MQ/00002.wav']

In [53]:
for i in b_list:
    c = glob.glob(i+'/*')
    for j in c:
        label = j.split('/')[-3]
        out_path = '/nvme/zhiyong/fllearn_data/G6/'
        generate_wave(j, out_path+label+'_'+j.split('/')[-2]+'_'+j.split('/')[-1])
        with open(out_path+'train_list_G6.txt', 'a') as f:
            f.write(label+' '+out_path+label+'_'+j.split('/')[-2]+'_'+j.split('/')[-1]+'\n')

/tmp/ipykernel_43156/3403348603.py:50: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  float_types = [float, np.float, np.float32, np.float64]


# G1 TEST VER

In [54]:
import glob
a = glob.glob('/nvme/zhiyong/voxceleb/vox1/test/wav/*')

In [55]:
b = a

In [56]:
b_list = []
for i in b:
    b_list.extend(glob.glob(i+'/*'))

In [57]:
b_list

['/nvme/zhiyong/voxceleb/vox1/test/wav/id10301/AeRSD9jIdSg',
 '/nvme/zhiyong/voxceleb/vox1/test/wav/id10301/rXRbmL7nzIo',
 '/nvme/zhiyong/voxceleb/vox1/test/wav/id10301/6uqGkiR2oTI',
 '/nvme/zhiyong/voxceleb/vox1/test/wav/id10301/UkrkPwZd77Y',
 '/nvme/zhiyong/voxceleb/vox1/test/wav/id10301/BVFTRGHuk3M',
 '/nvme/zhiyong/voxceleb/vox1/test/wav/id10301/Eu8Bj1gx5_4',
 '/nvme/zhiyong/voxceleb/vox1/test/wav/id10301/vhzF7dhGJzI',
 '/nvme/zhiyong/voxceleb/vox1/test/wav/id10301/eVfmXU0RtRo',
 '/nvme/zhiyong/voxceleb/vox1/test/wav/id10301/VIAnMw7J5tI',
 '/nvme/zhiyong/voxceleb/vox1/test/wav/id10301/hX6LjjVtnGs',
 '/nvme/zhiyong/voxceleb/vox1/test/wav/id10274/wH6ebv-K8AA',
 '/nvme/zhiyong/voxceleb/vox1/test/wav/id10274/YCIIiIR9B60',
 '/nvme/zhiyong/voxceleb/vox1/test/wav/id10274/7rkVfW2yw0c',
 '/nvme/zhiyong/voxceleb/vox1/test/wav/id10274/xbGp2KSUr0E',
 '/nvme/zhiyong/voxceleb/vox1/test/wav/id10274/89UnsytX6Y8',
 '/nvme/zhiyong/voxceleb/vox1/test/wav/id10274/L9q6XabEpEI',
 '/nvme/zhiyong/voxceleb

In [58]:
def generate_wave(path, out_path):
    absor, size_coef = room_properties('high', 'small')
    room = pra.ShoeBox(
        p=size_coef*np.array([4, 3, 2.5]),
        materials=pra.Material(absor),
        fs=16000,
        max_order=3,
        ray_tracing=True,
        air_absorption=True,
    )
    room.set_ray_tracing(receiver_radius=0.1, n_rays=10000, energy_thres=1e-5)

    # specify signal source
    fs, signal = wavfile.read(path)

    # add source to 2D room
    room.add_source([1.,1.,1.], signal=signal)

    # add microphone array
    room.add_microphone(loc=[3.5,2.5,1.])

    room.simulate()
    
    to_wav(out_path, fs, room.mic_array.signals[0,:], norm=True, bitdepth=np.int16)

In [59]:
glob.glob('/nvme/zhiyong/voxceleb/vox1/test/wav/id10301/AeRSD9jIdSg/*')

['/nvme/zhiyong/voxceleb/vox1/test/wav/id10301/AeRSD9jIdSg/00010.wav',
 '/nvme/zhiyong/voxceleb/vox1/test/wav/id10301/AeRSD9jIdSg/00004.wav',
 '/nvme/zhiyong/voxceleb/vox1/test/wav/id10301/AeRSD9jIdSg/00001.wav',
 '/nvme/zhiyong/voxceleb/vox1/test/wav/id10301/AeRSD9jIdSg/00006.wav',
 '/nvme/zhiyong/voxceleb/vox1/test/wav/id10301/AeRSD9jIdSg/00005.wav',
 '/nvme/zhiyong/voxceleb/vox1/test/wav/id10301/AeRSD9jIdSg/00009.wav',
 '/nvme/zhiyong/voxceleb/vox1/test/wav/id10301/AeRSD9jIdSg/00007.wav',
 '/nvme/zhiyong/voxceleb/vox1/test/wav/id10301/AeRSD9jIdSg/00008.wav',
 '/nvme/zhiyong/voxceleb/vox1/test/wav/id10301/AeRSD9jIdSg/00003.wav',
 '/nvme/zhiyong/voxceleb/vox1/test/wav/id10301/AeRSD9jIdSg/00002.wav']

In [60]:
for i in b_list:
    c = glob.glob(i+'/*')
    for j in c:
        label = j.split('/')[-3]
        out_path = '/nvme/zhiyong/fllearn_data/G1_VER/'
        generate_wave(j, out_path+label+'_'+j.split('/')[-2]+'_'+j.split('/')[-1])

/tmp/ipykernel_43156/3403348603.py:50: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  float_types = [float, np.float, np.float32, np.float64]


## create verlist

In [43]:
ver_ori_path = '/nvme/zhiyong/sdsv21/vox_o_triallist.txt'

In [44]:
with open(ver_ori_path, 'r') as f:
    ver_ori = f.readlines()

In [45]:
ver_ori

['1 vox1/test/wav/id10270/x6uYqmx31kE/00001.wav vox1/test/wav/id10270/8jEAjG6SegY/00008.wav\n',
 '0 vox1/test/wav/id10270/x6uYqmx31kE/00001.wav vox1/test/wav/id10300/ize_eiCFEg0/00003.wav\n',
 '1 vox1/test/wav/id10270/x6uYqmx31kE/00001.wav vox1/test/wav/id10270/GWXujl-xAVM/00017.wav\n',
 '0 vox1/test/wav/id10270/x6uYqmx31kE/00001.wav vox1/test/wav/id10273/0OCW1HUxZyg/00001.wav\n',
 '1 vox1/test/wav/id10270/x6uYqmx31kE/00001.wav vox1/test/wav/id10270/8jEAjG6SegY/00022.wav\n',
 '0 vox1/test/wav/id10270/x6uYqmx31kE/00001.wav vox1/test/wav/id10284/Uzxv7Axh3Z8/00001.wav\n',
 '1 vox1/test/wav/id10270/x6uYqmx31kE/00001.wav vox1/test/wav/id10270/GWXujl-xAVM/00033.wav\n',
 '0 vox1/test/wav/id10270/x6uYqmx31kE/00001.wav vox1/test/wav/id10284/7yx9A0yzLYk/00029.wav\n',
 '1 vox1/test/wav/id10270/x6uYqmx31kE/00002.wav vox1/test/wav/id10270/5r0dWxy17C8/00026.wav\n',
 '0 vox1/test/wav/id10270/x6uYqmx31kE/00002.wav vox1/test/wav/id10285/m-uILToQ9ss/00009.wav\n',
 '1 vox1/test/wav/id10270/x6uYqmx31kE/00

In [46]:
ver_new = []
for i in ver_ori:
    i = i.strip()
    ver_new.append(i.split(' ')[0]+' '
                'G1_VER/'+i.split(' ')[1].split('/')[-3]+'_'+i.split(' ')[1].split('/')[-2]+'_'+i.split(' ')[1].split('/')[-1]+' '
                'G1_VER/'+i.split(' ')[2].split('/')[-3]+'_'+i.split(' ')[2].split('/')[-2]+'_'+i.split(' ')[2].split('/')[-1]+'\n')

In [47]:
ver_new

['1 G1_VER/id10270_x6uYqmx31kE_00001.wav G1_VER/id10270_8jEAjG6SegY_00008.wav\n',
 '0 G1_VER/id10270_x6uYqmx31kE_00001.wav G1_VER/id10300_ize_eiCFEg0_00003.wav\n',
 '1 G1_VER/id10270_x6uYqmx31kE_00001.wav G1_VER/id10270_GWXujl-xAVM_00017.wav\n',
 '0 G1_VER/id10270_x6uYqmx31kE_00001.wav G1_VER/id10273_0OCW1HUxZyg_00001.wav\n',
 '1 G1_VER/id10270_x6uYqmx31kE_00001.wav G1_VER/id10270_8jEAjG6SegY_00022.wav\n',
 '0 G1_VER/id10270_x6uYqmx31kE_00001.wav G1_VER/id10284_Uzxv7Axh3Z8_00001.wav\n',
 '1 G1_VER/id10270_x6uYqmx31kE_00001.wav G1_VER/id10270_GWXujl-xAVM_00033.wav\n',
 '0 G1_VER/id10270_x6uYqmx31kE_00001.wav G1_VER/id10284_7yx9A0yzLYk_00029.wav\n',
 '1 G1_VER/id10270_x6uYqmx31kE_00002.wav G1_VER/id10270_5r0dWxy17C8_00026.wav\n',
 '0 G1_VER/id10270_x6uYqmx31kE_00002.wav G1_VER/id10285_m-uILToQ9ss_00009.wav\n',
 '1 G1_VER/id10270_x6uYqmx31kE_00002.wav G1_VER/id10270_GWXujl-xAVM_00035.wav\n',
 '0 G1_VER/id10270_x6uYqmx31kE_00002.wav G1_VER/id10306_uzt36PBzT2w_00001.wav\n',
 '1 G1_VER/id102

In [48]:
# write ver_new to .txt file
with open('/nvme/zhiyong/fllearn_data/ver_list_G1_VER.txt', 'w') as f:
    for i in ver_new:
        f.write(i)

# G2 TEST VER

In [61]:
import glob
a = glob.glob('/nvme/zhiyong/voxceleb/vox1/test/wav/*')

In [62]:
b = a

In [63]:
b_list = []
for i in b:
    b_list.extend(glob.glob(i+'/*'))

In [64]:
b_list

['/nvme/zhiyong/voxceleb/vox1/test/wav/id10301/AeRSD9jIdSg',
 '/nvme/zhiyong/voxceleb/vox1/test/wav/id10301/rXRbmL7nzIo',
 '/nvme/zhiyong/voxceleb/vox1/test/wav/id10301/6uqGkiR2oTI',
 '/nvme/zhiyong/voxceleb/vox1/test/wav/id10301/UkrkPwZd77Y',
 '/nvme/zhiyong/voxceleb/vox1/test/wav/id10301/BVFTRGHuk3M',
 '/nvme/zhiyong/voxceleb/vox1/test/wav/id10301/Eu8Bj1gx5_4',
 '/nvme/zhiyong/voxceleb/vox1/test/wav/id10301/vhzF7dhGJzI',
 '/nvme/zhiyong/voxceleb/vox1/test/wav/id10301/eVfmXU0RtRo',
 '/nvme/zhiyong/voxceleb/vox1/test/wav/id10301/VIAnMw7J5tI',
 '/nvme/zhiyong/voxceleb/vox1/test/wav/id10301/hX6LjjVtnGs',
 '/nvme/zhiyong/voxceleb/vox1/test/wav/id10274/wH6ebv-K8AA',
 '/nvme/zhiyong/voxceleb/vox1/test/wav/id10274/YCIIiIR9B60',
 '/nvme/zhiyong/voxceleb/vox1/test/wav/id10274/7rkVfW2yw0c',
 '/nvme/zhiyong/voxceleb/vox1/test/wav/id10274/xbGp2KSUr0E',
 '/nvme/zhiyong/voxceleb/vox1/test/wav/id10274/89UnsytX6Y8',
 '/nvme/zhiyong/voxceleb/vox1/test/wav/id10274/L9q6XabEpEI',
 '/nvme/zhiyong/voxceleb

In [65]:
def generate_wave(path, out_path):
    absor, size_coef = room_properties('low', 'large')
    room = pra.ShoeBox(
        p=np.array([4*size_coef, 3*size_coef, 2.5]),
        materials=pra.Material(absor),
        fs=16000,
        max_order=3,
        ray_tracing=True,
        air_absorption=True,
    )
    room.set_ray_tracing(receiver_radius=0.1, n_rays=10000, energy_thres=1e-5)

    # specify signal source
    fs, signal = wavfile.read(path)

    # add source to 2D room
    room.add_source([1.,1.,1.], signal=signal)

    # add microphone array
    room.add_microphone(loc=[17.,12.,1.])

    room.simulate()
    to_wav(out_path, fs, room.mic_array.signals[0,:], norm=True, bitdepth=np.int16)

In [66]:
glob.glob('/nvme/zhiyong/voxceleb/vox1/test/wav/id10301/AeRSD9jIdSg/*')

['/nvme/zhiyong/voxceleb/vox1/test/wav/id10301/AeRSD9jIdSg/00010.wav',
 '/nvme/zhiyong/voxceleb/vox1/test/wav/id10301/AeRSD9jIdSg/00004.wav',
 '/nvme/zhiyong/voxceleb/vox1/test/wav/id10301/AeRSD9jIdSg/00001.wav',
 '/nvme/zhiyong/voxceleb/vox1/test/wav/id10301/AeRSD9jIdSg/00006.wav',
 '/nvme/zhiyong/voxceleb/vox1/test/wav/id10301/AeRSD9jIdSg/00005.wav',
 '/nvme/zhiyong/voxceleb/vox1/test/wav/id10301/AeRSD9jIdSg/00009.wav',
 '/nvme/zhiyong/voxceleb/vox1/test/wav/id10301/AeRSD9jIdSg/00007.wav',
 '/nvme/zhiyong/voxceleb/vox1/test/wav/id10301/AeRSD9jIdSg/00008.wav',
 '/nvme/zhiyong/voxceleb/vox1/test/wav/id10301/AeRSD9jIdSg/00003.wav',
 '/nvme/zhiyong/voxceleb/vox1/test/wav/id10301/AeRSD9jIdSg/00002.wav']

In [67]:
for i in b_list:
    c = glob.glob(i+'/*')
    for j in c:
        label = j.split('/')[-3]
        out_path = '/nvme/zhiyong/fllearn_data/G2_VER/'
        generate_wave(j, out_path+label+'_'+j.split('/')[-2]+'_'+j.split('/')[-1])

/tmp/ipykernel_43156/3403348603.py:50: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  float_types = [float, np.float, np.float32, np.float64]


## create verlist

In [49]:
ver_ori_path = '/nvme/zhiyong/sdsv21/vox_o_triallist.txt'

In [50]:
with open(ver_ori_path, 'r') as f:
    ver_ori = f.readlines()

In [51]:
ver_ori

['1 vox1/test/wav/id10270/x6uYqmx31kE/00001.wav vox1/test/wav/id10270/8jEAjG6SegY/00008.wav\n',
 '0 vox1/test/wav/id10270/x6uYqmx31kE/00001.wav vox1/test/wav/id10300/ize_eiCFEg0/00003.wav\n',
 '1 vox1/test/wav/id10270/x6uYqmx31kE/00001.wav vox1/test/wav/id10270/GWXujl-xAVM/00017.wav\n',
 '0 vox1/test/wav/id10270/x6uYqmx31kE/00001.wav vox1/test/wav/id10273/0OCW1HUxZyg/00001.wav\n',
 '1 vox1/test/wav/id10270/x6uYqmx31kE/00001.wav vox1/test/wav/id10270/8jEAjG6SegY/00022.wav\n',
 '0 vox1/test/wav/id10270/x6uYqmx31kE/00001.wav vox1/test/wav/id10284/Uzxv7Axh3Z8/00001.wav\n',
 '1 vox1/test/wav/id10270/x6uYqmx31kE/00001.wav vox1/test/wav/id10270/GWXujl-xAVM/00033.wav\n',
 '0 vox1/test/wav/id10270/x6uYqmx31kE/00001.wav vox1/test/wav/id10284/7yx9A0yzLYk/00029.wav\n',
 '1 vox1/test/wav/id10270/x6uYqmx31kE/00002.wav vox1/test/wav/id10270/5r0dWxy17C8/00026.wav\n',
 '0 vox1/test/wav/id10270/x6uYqmx31kE/00002.wav vox1/test/wav/id10285/m-uILToQ9ss/00009.wav\n',
 '1 vox1/test/wav/id10270/x6uYqmx31kE/00

In [52]:
ver_new = []
for i in ver_ori:
    i = i.strip()
    ver_new.append(i.split(' ')[0]+' '
                'G2_VER/'+i.split(' ')[1].split('/')[-3]+'_'+i.split(' ')[1].split('/')[-2]+'_'+i.split(' ')[1].split('/')[-1]+' '
                'G2_VER/'+i.split(' ')[2].split('/')[-3]+'_'+i.split(' ')[2].split('/')[-2]+'_'+i.split(' ')[2].split('/')[-1]+'\n')

In [53]:
ver_new

['1 G2_VER/id10270_x6uYqmx31kE_00001.wav G2_VER/id10270_8jEAjG6SegY_00008.wav\n',
 '0 G2_VER/id10270_x6uYqmx31kE_00001.wav G2_VER/id10300_ize_eiCFEg0_00003.wav\n',
 '1 G2_VER/id10270_x6uYqmx31kE_00001.wav G2_VER/id10270_GWXujl-xAVM_00017.wav\n',
 '0 G2_VER/id10270_x6uYqmx31kE_00001.wav G2_VER/id10273_0OCW1HUxZyg_00001.wav\n',
 '1 G2_VER/id10270_x6uYqmx31kE_00001.wav G2_VER/id10270_8jEAjG6SegY_00022.wav\n',
 '0 G2_VER/id10270_x6uYqmx31kE_00001.wav G2_VER/id10284_Uzxv7Axh3Z8_00001.wav\n',
 '1 G2_VER/id10270_x6uYqmx31kE_00001.wav G2_VER/id10270_GWXujl-xAVM_00033.wav\n',
 '0 G2_VER/id10270_x6uYqmx31kE_00001.wav G2_VER/id10284_7yx9A0yzLYk_00029.wav\n',
 '1 G2_VER/id10270_x6uYqmx31kE_00002.wav G2_VER/id10270_5r0dWxy17C8_00026.wav\n',
 '0 G2_VER/id10270_x6uYqmx31kE_00002.wav G2_VER/id10285_m-uILToQ9ss_00009.wav\n',
 '1 G2_VER/id10270_x6uYqmx31kE_00002.wav G2_VER/id10270_GWXujl-xAVM_00035.wav\n',
 '0 G2_VER/id10270_x6uYqmx31kE_00002.wav G2_VER/id10306_uzt36PBzT2w_00001.wav\n',
 '1 G2_VER/id102

In [54]:
# write ver_new to .txt file
with open('/nvme/zhiyong/fllearn_data/ver_list_G2_VER.txt', 'w') as f:
    for i in ver_new:
        f.write(i)

# G3 TEST VER

In [68]:
import glob
a = glob.glob('/nvme/zhiyong/voxceleb/vox1/test/wav/*')

In [69]:
b = a

In [70]:
b_list = []
for i in b:
    b_list.extend(glob.glob(i+'/*'))

In [71]:
b_list

['/nvme/zhiyong/voxceleb/vox1/test/wav/id10301/AeRSD9jIdSg',
 '/nvme/zhiyong/voxceleb/vox1/test/wav/id10301/rXRbmL7nzIo',
 '/nvme/zhiyong/voxceleb/vox1/test/wav/id10301/6uqGkiR2oTI',
 '/nvme/zhiyong/voxceleb/vox1/test/wav/id10301/UkrkPwZd77Y',
 '/nvme/zhiyong/voxceleb/vox1/test/wav/id10301/BVFTRGHuk3M',
 '/nvme/zhiyong/voxceleb/vox1/test/wav/id10301/Eu8Bj1gx5_4',
 '/nvme/zhiyong/voxceleb/vox1/test/wav/id10301/vhzF7dhGJzI',
 '/nvme/zhiyong/voxceleb/vox1/test/wav/id10301/eVfmXU0RtRo',
 '/nvme/zhiyong/voxceleb/vox1/test/wav/id10301/VIAnMw7J5tI',
 '/nvme/zhiyong/voxceleb/vox1/test/wav/id10301/hX6LjjVtnGs',
 '/nvme/zhiyong/voxceleb/vox1/test/wav/id10274/wH6ebv-K8AA',
 '/nvme/zhiyong/voxceleb/vox1/test/wav/id10274/YCIIiIR9B60',
 '/nvme/zhiyong/voxceleb/vox1/test/wav/id10274/7rkVfW2yw0c',
 '/nvme/zhiyong/voxceleb/vox1/test/wav/id10274/xbGp2KSUr0E',
 '/nvme/zhiyong/voxceleb/vox1/test/wav/id10274/89UnsytX6Y8',
 '/nvme/zhiyong/voxceleb/vox1/test/wav/id10274/L9q6XabEpEI',
 '/nvme/zhiyong/voxceleb

In [72]:
def generate_wave(path, out_path):
    absor, size_coef = room_properties('medium', 'medium')
    room = pra.ShoeBox(
        p=np.array([4*size_coef, 3*size_coef, 2.5]),
        materials=pra.Material(absor),
        fs=16000,
        max_order=3,
        ray_tracing=True,
        air_absorption=True,
    )
    room.set_ray_tracing(receiver_radius=0.1, n_rays=10000, energy_thres=1e-5)

    # specify signal source
    fs, signal = wavfile.read(path)

    # add source to 2D room
    room.add_source([1.,1.,1.], signal=signal)

    # add microphone array
    room.add_microphone(loc=[8.,6.,1.])

    room.simulate()
    to_wav(out_path, fs, room.mic_array.signals[0,:], norm=True, bitdepth=np.int16)

In [73]:
glob.glob('/nvme/zhiyong/voxceleb/vox1/test/wav/id10301/AeRSD9jIdSg/*')

['/nvme/zhiyong/voxceleb/vox1/test/wav/id10301/AeRSD9jIdSg/00010.wav',
 '/nvme/zhiyong/voxceleb/vox1/test/wav/id10301/AeRSD9jIdSg/00004.wav',
 '/nvme/zhiyong/voxceleb/vox1/test/wav/id10301/AeRSD9jIdSg/00001.wav',
 '/nvme/zhiyong/voxceleb/vox1/test/wav/id10301/AeRSD9jIdSg/00006.wav',
 '/nvme/zhiyong/voxceleb/vox1/test/wav/id10301/AeRSD9jIdSg/00005.wav',
 '/nvme/zhiyong/voxceleb/vox1/test/wav/id10301/AeRSD9jIdSg/00009.wav',
 '/nvme/zhiyong/voxceleb/vox1/test/wav/id10301/AeRSD9jIdSg/00007.wav',
 '/nvme/zhiyong/voxceleb/vox1/test/wav/id10301/AeRSD9jIdSg/00008.wav',
 '/nvme/zhiyong/voxceleb/vox1/test/wav/id10301/AeRSD9jIdSg/00003.wav',
 '/nvme/zhiyong/voxceleb/vox1/test/wav/id10301/AeRSD9jIdSg/00002.wav']

In [74]:
for i in b_list:
    c = glob.glob(i+'/*')
    for j in c:
        label = j.split('/')[-3]
        out_path = '/nvme/zhiyong/fllearn_data/G3_VER/'
        generate_wave(j, out_path+label+'_'+j.split('/')[-2]+'_'+j.split('/')[-1])

/tmp/ipykernel_43156/3403348603.py:50: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  float_types = [float, np.float, np.float32, np.float64]


In [55]:
ver_ori_path = '/nvme/zhiyong/sdsv21/vox_o_triallist.txt'

In [56]:
with open(ver_ori_path, 'r') as f:
    ver_ori = f.readlines()

In [57]:
ver_ori

['1 vox1/test/wav/id10270/x6uYqmx31kE/00001.wav vox1/test/wav/id10270/8jEAjG6SegY/00008.wav\n',
 '0 vox1/test/wav/id10270/x6uYqmx31kE/00001.wav vox1/test/wav/id10300/ize_eiCFEg0/00003.wav\n',
 '1 vox1/test/wav/id10270/x6uYqmx31kE/00001.wav vox1/test/wav/id10270/GWXujl-xAVM/00017.wav\n',
 '0 vox1/test/wav/id10270/x6uYqmx31kE/00001.wav vox1/test/wav/id10273/0OCW1HUxZyg/00001.wav\n',
 '1 vox1/test/wav/id10270/x6uYqmx31kE/00001.wav vox1/test/wav/id10270/8jEAjG6SegY/00022.wav\n',
 '0 vox1/test/wav/id10270/x6uYqmx31kE/00001.wav vox1/test/wav/id10284/Uzxv7Axh3Z8/00001.wav\n',
 '1 vox1/test/wav/id10270/x6uYqmx31kE/00001.wav vox1/test/wav/id10270/GWXujl-xAVM/00033.wav\n',
 '0 vox1/test/wav/id10270/x6uYqmx31kE/00001.wav vox1/test/wav/id10284/7yx9A0yzLYk/00029.wav\n',
 '1 vox1/test/wav/id10270/x6uYqmx31kE/00002.wav vox1/test/wav/id10270/5r0dWxy17C8/00026.wav\n',
 '0 vox1/test/wav/id10270/x6uYqmx31kE/00002.wav vox1/test/wav/id10285/m-uILToQ9ss/00009.wav\n',
 '1 vox1/test/wav/id10270/x6uYqmx31kE/00

In [58]:
ver_new = []
for i in ver_ori:
    i = i.strip()
    ver_new.append(i.split(' ')[0]+' '
                'G3_VER/'+i.split(' ')[1].split('/')[-3]+'_'+i.split(' ')[1].split('/')[-2]+'_'+i.split(' ')[1].split('/')[-1]+' '
                'G3_VER/'+i.split(' ')[2].split('/')[-3]+'_'+i.split(' ')[2].split('/')[-2]+'_'+i.split(' ')[2].split('/')[-1]+'\n')

In [59]:
ver_new

['1 G3_VER/id10270_x6uYqmx31kE_00001.wav G3_VER/id10270_8jEAjG6SegY_00008.wav\n',
 '0 G3_VER/id10270_x6uYqmx31kE_00001.wav G3_VER/id10300_ize_eiCFEg0_00003.wav\n',
 '1 G3_VER/id10270_x6uYqmx31kE_00001.wav G3_VER/id10270_GWXujl-xAVM_00017.wav\n',
 '0 G3_VER/id10270_x6uYqmx31kE_00001.wav G3_VER/id10273_0OCW1HUxZyg_00001.wav\n',
 '1 G3_VER/id10270_x6uYqmx31kE_00001.wav G3_VER/id10270_8jEAjG6SegY_00022.wav\n',
 '0 G3_VER/id10270_x6uYqmx31kE_00001.wav G3_VER/id10284_Uzxv7Axh3Z8_00001.wav\n',
 '1 G3_VER/id10270_x6uYqmx31kE_00001.wav G3_VER/id10270_GWXujl-xAVM_00033.wav\n',
 '0 G3_VER/id10270_x6uYqmx31kE_00001.wav G3_VER/id10284_7yx9A0yzLYk_00029.wav\n',
 '1 G3_VER/id10270_x6uYqmx31kE_00002.wav G3_VER/id10270_5r0dWxy17C8_00026.wav\n',
 '0 G3_VER/id10270_x6uYqmx31kE_00002.wav G3_VER/id10285_m-uILToQ9ss_00009.wav\n',
 '1 G3_VER/id10270_x6uYqmx31kE_00002.wav G3_VER/id10270_GWXujl-xAVM_00035.wav\n',
 '0 G3_VER/id10270_x6uYqmx31kE_00002.wav G3_VER/id10306_uzt36PBzT2w_00001.wav\n',
 '1 G3_VER/id102

In [60]:
# write ver_new to .txt file
with open('/nvme/zhiyong/fllearn_data/ver_list_G3_VER.txt', 'w') as f:
    for i in ver_new:
        f.write(i)

# G4 TEST VER

In [75]:
import glob
a = glob.glob('/nvme/zhiyong/voxceleb/vox1/test/wav/*')

In [76]:
b = a

In [77]:
b_list = []
for i in b:
    b_list.extend(glob.glob(i+'/*'))

In [78]:
b_list

['/nvme/zhiyong/voxceleb/vox1/test/wav/id10301/AeRSD9jIdSg',
 '/nvme/zhiyong/voxceleb/vox1/test/wav/id10301/rXRbmL7nzIo',
 '/nvme/zhiyong/voxceleb/vox1/test/wav/id10301/6uqGkiR2oTI',
 '/nvme/zhiyong/voxceleb/vox1/test/wav/id10301/UkrkPwZd77Y',
 '/nvme/zhiyong/voxceleb/vox1/test/wav/id10301/BVFTRGHuk3M',
 '/nvme/zhiyong/voxceleb/vox1/test/wav/id10301/Eu8Bj1gx5_4',
 '/nvme/zhiyong/voxceleb/vox1/test/wav/id10301/vhzF7dhGJzI',
 '/nvme/zhiyong/voxceleb/vox1/test/wav/id10301/eVfmXU0RtRo',
 '/nvme/zhiyong/voxceleb/vox1/test/wav/id10301/VIAnMw7J5tI',
 '/nvme/zhiyong/voxceleb/vox1/test/wav/id10301/hX6LjjVtnGs',
 '/nvme/zhiyong/voxceleb/vox1/test/wav/id10274/wH6ebv-K8AA',
 '/nvme/zhiyong/voxceleb/vox1/test/wav/id10274/YCIIiIR9B60',
 '/nvme/zhiyong/voxceleb/vox1/test/wav/id10274/7rkVfW2yw0c',
 '/nvme/zhiyong/voxceleb/vox1/test/wav/id10274/xbGp2KSUr0E',
 '/nvme/zhiyong/voxceleb/vox1/test/wav/id10274/89UnsytX6Y8',
 '/nvme/zhiyong/voxceleb/vox1/test/wav/id10274/L9q6XabEpEI',
 '/nvme/zhiyong/voxceleb

In [79]:
def generate_wave(path, out_path):
    absor, size_coef = room_properties('medium', 'medium')
    room = pra.ShoeBox(
        p=np.array([4*size_coef, 3*size_coef, 2.5]),
        materials=pra.Material(absor),
        fs=16000,
        max_order=3,
        ray_tracing=True,
        air_absorption=True,
    )
    room.set_ray_tracing(receiver_radius=0.1, n_rays=10000, energy_thres=1e-5)

    # specify signal source
    fs, signal = wavfile.read(path)

    # add source to 2D room
    room.add_source([1.,1.,1.], signal=signal)

    # add noise source
    fs, noise = wavfile.read("/nvme/zhiyong/musan/noise/free-sound/noise-free-sound-0000.wav")

    room.add_source([4.,7.,1.], signal=noise[:len(signal)], delay=0.)

    # add microphone array
    room.add_microphone(loc=[8.,4.,1.])

    room.simulate()
    to_wav(out_path, fs, room.mic_array.signals[0,:], norm=True, bitdepth=np.int16)

In [80]:
glob.glob('/nvme/zhiyong/voxceleb/vox1/test/wav/id10301/AeRSD9jIdSg/*')

['/nvme/zhiyong/voxceleb/vox1/test/wav/id10301/AeRSD9jIdSg/00010.wav',
 '/nvme/zhiyong/voxceleb/vox1/test/wav/id10301/AeRSD9jIdSg/00004.wav',
 '/nvme/zhiyong/voxceleb/vox1/test/wav/id10301/AeRSD9jIdSg/00001.wav',
 '/nvme/zhiyong/voxceleb/vox1/test/wav/id10301/AeRSD9jIdSg/00006.wav',
 '/nvme/zhiyong/voxceleb/vox1/test/wav/id10301/AeRSD9jIdSg/00005.wav',
 '/nvme/zhiyong/voxceleb/vox1/test/wav/id10301/AeRSD9jIdSg/00009.wav',
 '/nvme/zhiyong/voxceleb/vox1/test/wav/id10301/AeRSD9jIdSg/00007.wav',
 '/nvme/zhiyong/voxceleb/vox1/test/wav/id10301/AeRSD9jIdSg/00008.wav',
 '/nvme/zhiyong/voxceleb/vox1/test/wav/id10301/AeRSD9jIdSg/00003.wav',
 '/nvme/zhiyong/voxceleb/vox1/test/wav/id10301/AeRSD9jIdSg/00002.wav']

In [81]:
for i in b_list:
    c = glob.glob(i+'/*')
    for j in c:
        label = j.split('/')[-3]
        out_path = '/nvme/zhiyong/fllearn_data/G4_VER/'
        generate_wave(j, out_path+label+'_'+j.split('/')[-2]+'_'+j.split('/')[-1])

/tmp/ipykernel_43156/3403348603.py:50: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  float_types = [float, np.float, np.float32, np.float64]


In [61]:
ver_ori_path = '/nvme/zhiyong/sdsv21/vox_o_triallist.txt'

In [62]:
with open(ver_ori_path, 'r') as f:
    ver_ori = f.readlines()

In [63]:
ver_ori

['1 vox1/test/wav/id10270/x6uYqmx31kE/00001.wav vox1/test/wav/id10270/8jEAjG6SegY/00008.wav\n',
 '0 vox1/test/wav/id10270/x6uYqmx31kE/00001.wav vox1/test/wav/id10300/ize_eiCFEg0/00003.wav\n',
 '1 vox1/test/wav/id10270/x6uYqmx31kE/00001.wav vox1/test/wav/id10270/GWXujl-xAVM/00017.wav\n',
 '0 vox1/test/wav/id10270/x6uYqmx31kE/00001.wav vox1/test/wav/id10273/0OCW1HUxZyg/00001.wav\n',
 '1 vox1/test/wav/id10270/x6uYqmx31kE/00001.wav vox1/test/wav/id10270/8jEAjG6SegY/00022.wav\n',
 '0 vox1/test/wav/id10270/x6uYqmx31kE/00001.wav vox1/test/wav/id10284/Uzxv7Axh3Z8/00001.wav\n',
 '1 vox1/test/wav/id10270/x6uYqmx31kE/00001.wav vox1/test/wav/id10270/GWXujl-xAVM/00033.wav\n',
 '0 vox1/test/wav/id10270/x6uYqmx31kE/00001.wav vox1/test/wav/id10284/7yx9A0yzLYk/00029.wav\n',
 '1 vox1/test/wav/id10270/x6uYqmx31kE/00002.wav vox1/test/wav/id10270/5r0dWxy17C8/00026.wav\n',
 '0 vox1/test/wav/id10270/x6uYqmx31kE/00002.wav vox1/test/wav/id10285/m-uILToQ9ss/00009.wav\n',
 '1 vox1/test/wav/id10270/x6uYqmx31kE/00

In [64]:
ver_new = []
for i in ver_ori:
    i = i.strip()
    ver_new.append(i.split(' ')[0]+' '
                'G4_VER/'+i.split(' ')[1].split('/')[-3]+'_'+i.split(' ')[1].split('/')[-2]+'_'+i.split(' ')[1].split('/')[-1]+' '
                'G4_VER/'+i.split(' ')[2].split('/')[-3]+'_'+i.split(' ')[2].split('/')[-2]+'_'+i.split(' ')[2].split('/')[-1]+'\n')

In [65]:
ver_new

['1 G4_VER/id10270_x6uYqmx31kE_00001.wav G4_VER/id10270_8jEAjG6SegY_00008.wav\n',
 '0 G4_VER/id10270_x6uYqmx31kE_00001.wav G4_VER/id10300_ize_eiCFEg0_00003.wav\n',
 '1 G4_VER/id10270_x6uYqmx31kE_00001.wav G4_VER/id10270_GWXujl-xAVM_00017.wav\n',
 '0 G4_VER/id10270_x6uYqmx31kE_00001.wav G4_VER/id10273_0OCW1HUxZyg_00001.wav\n',
 '1 G4_VER/id10270_x6uYqmx31kE_00001.wav G4_VER/id10270_8jEAjG6SegY_00022.wav\n',
 '0 G4_VER/id10270_x6uYqmx31kE_00001.wav G4_VER/id10284_Uzxv7Axh3Z8_00001.wav\n',
 '1 G4_VER/id10270_x6uYqmx31kE_00001.wav G4_VER/id10270_GWXujl-xAVM_00033.wav\n',
 '0 G4_VER/id10270_x6uYqmx31kE_00001.wav G4_VER/id10284_7yx9A0yzLYk_00029.wav\n',
 '1 G4_VER/id10270_x6uYqmx31kE_00002.wav G4_VER/id10270_5r0dWxy17C8_00026.wav\n',
 '0 G4_VER/id10270_x6uYqmx31kE_00002.wav G4_VER/id10285_m-uILToQ9ss_00009.wav\n',
 '1 G4_VER/id10270_x6uYqmx31kE_00002.wav G4_VER/id10270_GWXujl-xAVM_00035.wav\n',
 '0 G4_VER/id10270_x6uYqmx31kE_00002.wav G4_VER/id10306_uzt36PBzT2w_00001.wav\n',
 '1 G4_VER/id102

In [66]:
# write ver_new to .txt file
with open('/nvme/zhiyong/fllearn_data/ver_list_G4_VER.txt', 'w') as f:
    for i in ver_new:
        f.write(i)

# G5 TEST VER

In [82]:
import glob
a = glob.glob('/nvme/zhiyong/voxceleb/vox1/test/wav/*')

In [83]:
b = a

In [84]:
b_list = []
for i in b:
    b_list.extend(glob.glob(i+'/*'))

In [85]:
b_list

['/nvme/zhiyong/voxceleb/vox1/test/wav/id10301/AeRSD9jIdSg',
 '/nvme/zhiyong/voxceleb/vox1/test/wav/id10301/rXRbmL7nzIo',
 '/nvme/zhiyong/voxceleb/vox1/test/wav/id10301/6uqGkiR2oTI',
 '/nvme/zhiyong/voxceleb/vox1/test/wav/id10301/UkrkPwZd77Y',
 '/nvme/zhiyong/voxceleb/vox1/test/wav/id10301/BVFTRGHuk3M',
 '/nvme/zhiyong/voxceleb/vox1/test/wav/id10301/Eu8Bj1gx5_4',
 '/nvme/zhiyong/voxceleb/vox1/test/wav/id10301/vhzF7dhGJzI',
 '/nvme/zhiyong/voxceleb/vox1/test/wav/id10301/eVfmXU0RtRo',
 '/nvme/zhiyong/voxceleb/vox1/test/wav/id10301/VIAnMw7J5tI',
 '/nvme/zhiyong/voxceleb/vox1/test/wav/id10301/hX6LjjVtnGs',
 '/nvme/zhiyong/voxceleb/vox1/test/wav/id10274/wH6ebv-K8AA',
 '/nvme/zhiyong/voxceleb/vox1/test/wav/id10274/YCIIiIR9B60',
 '/nvme/zhiyong/voxceleb/vox1/test/wav/id10274/7rkVfW2yw0c',
 '/nvme/zhiyong/voxceleb/vox1/test/wav/id10274/xbGp2KSUr0E',
 '/nvme/zhiyong/voxceleb/vox1/test/wav/id10274/89UnsytX6Y8',
 '/nvme/zhiyong/voxceleb/vox1/test/wav/id10274/L9q6XabEpEI',
 '/nvme/zhiyong/voxceleb

In [86]:
def generate_wave(path, out_path):
    fft_len = 512
    Lg_t = 0.100                # filter size in seconds
    Lg = np.ceil(Lg_t*16000)       # in samples
    absor, size_coef = room_properties('medium', 'medium')
    room = pra.ShoeBox(
        p=np.array([4*size_coef, 3*size_coef, 2.5]),
        materials=pra.Material(absor),
        fs=16000,
        max_order=3,
        ray_tracing=True,
        air_absorption=True,
    )
    room.set_ray_tracing(receiver_radius=0.1, n_rays=10000, energy_thres=1e-5)

    # specify signal source
    fs, signal = wavfile.read(path)

    # add source to 2D room
    room.add_source([1.,1.,1.], signal=signal)

    # add microphone array
    R = pra.circular_2D_array(center=[8.,2.], M=6, phi0=0, radius=0.05)
    R = np.concatenate((R, np.ones((1, 6))))
    # dir_list = []
    # for i in range(6):
    #     dir_list.append(CardioidFamily(
    #     orientation=DirectionVector(azimuth=0, colatitude=90, degrees=True),
    #     pattern_enum=DirectivityPattern.OMNI,))

    # mics = pra.MicrophoneArray(R, room.fs, directivity=dir_list)
    mics = pra.Beamformer(R, fs, N=fft_len, Lg=Lg)
    room.add_microphone_array(mics)

    # Compute DAS weights
    mics.rake_delay_and_sum_weights(room.sources[0][:1])

    room.simulate()
    signal_das = mics.process(FD=False)
    to_wav(out_path, fs, signal_das, norm=True, bitdepth=np.int16)

In [87]:
glob.glob('/nvme/zhiyong/voxceleb/vox1/test/wav/id10301/AeRSD9jIdSg/*')

['/nvme/zhiyong/voxceleb/vox1/test/wav/id10301/AeRSD9jIdSg/00010.wav',
 '/nvme/zhiyong/voxceleb/vox1/test/wav/id10301/AeRSD9jIdSg/00004.wav',
 '/nvme/zhiyong/voxceleb/vox1/test/wav/id10301/AeRSD9jIdSg/00001.wav',
 '/nvme/zhiyong/voxceleb/vox1/test/wav/id10301/AeRSD9jIdSg/00006.wav',
 '/nvme/zhiyong/voxceleb/vox1/test/wav/id10301/AeRSD9jIdSg/00005.wav',
 '/nvme/zhiyong/voxceleb/vox1/test/wav/id10301/AeRSD9jIdSg/00009.wav',
 '/nvme/zhiyong/voxceleb/vox1/test/wav/id10301/AeRSD9jIdSg/00007.wav',
 '/nvme/zhiyong/voxceleb/vox1/test/wav/id10301/AeRSD9jIdSg/00008.wav',
 '/nvme/zhiyong/voxceleb/vox1/test/wav/id10301/AeRSD9jIdSg/00003.wav',
 '/nvme/zhiyong/voxceleb/vox1/test/wav/id10301/AeRSD9jIdSg/00002.wav']

In [88]:
for i in b_list:
    c = glob.glob(i+'/*')
    for j in c:
        label = j.split('/')[-3]
        out_path = '/nvme/zhiyong/fllearn_data/G5_VER/'
        generate_wave(j, out_path+label+'_'+j.split('/')[-2]+'_'+j.split('/')[-1])

/tmp/ipykernel_43156/3403348603.py:50: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  float_types = [float, np.float, np.float32, np.float64]


In [67]:
ver_ori_path = '/nvme/zhiyong/sdsv21/vox_o_triallist.txt'

In [68]:
with open(ver_ori_path, 'r') as f:
    ver_ori = f.readlines()

In [69]:
ver_ori

['1 vox1/test/wav/id10270/x6uYqmx31kE/00001.wav vox1/test/wav/id10270/8jEAjG6SegY/00008.wav\n',
 '0 vox1/test/wav/id10270/x6uYqmx31kE/00001.wav vox1/test/wav/id10300/ize_eiCFEg0/00003.wav\n',
 '1 vox1/test/wav/id10270/x6uYqmx31kE/00001.wav vox1/test/wav/id10270/GWXujl-xAVM/00017.wav\n',
 '0 vox1/test/wav/id10270/x6uYqmx31kE/00001.wav vox1/test/wav/id10273/0OCW1HUxZyg/00001.wav\n',
 '1 vox1/test/wav/id10270/x6uYqmx31kE/00001.wav vox1/test/wav/id10270/8jEAjG6SegY/00022.wav\n',
 '0 vox1/test/wav/id10270/x6uYqmx31kE/00001.wav vox1/test/wav/id10284/Uzxv7Axh3Z8/00001.wav\n',
 '1 vox1/test/wav/id10270/x6uYqmx31kE/00001.wav vox1/test/wav/id10270/GWXujl-xAVM/00033.wav\n',
 '0 vox1/test/wav/id10270/x6uYqmx31kE/00001.wav vox1/test/wav/id10284/7yx9A0yzLYk/00029.wav\n',
 '1 vox1/test/wav/id10270/x6uYqmx31kE/00002.wav vox1/test/wav/id10270/5r0dWxy17C8/00026.wav\n',
 '0 vox1/test/wav/id10270/x6uYqmx31kE/00002.wav vox1/test/wav/id10285/m-uILToQ9ss/00009.wav\n',
 '1 vox1/test/wav/id10270/x6uYqmx31kE/00

In [70]:
ver_new = []
for i in ver_ori:
    i = i.strip()
    ver_new.append(i.split(' ')[0]+' '
                'G5_VER/'+i.split(' ')[1].split('/')[-3]+'_'+i.split(' ')[1].split('/')[-2]+'_'+i.split(' ')[1].split('/')[-1]+' '
                'G5_VER/'+i.split(' ')[2].split('/')[-3]+'_'+i.split(' ')[2].split('/')[-2]+'_'+i.split(' ')[2].split('/')[-1]+'\n')

In [71]:
ver_new

['1 G5_VER/id10270_x6uYqmx31kE_00001.wav G5_VER/id10270_8jEAjG6SegY_00008.wav\n',
 '0 G5_VER/id10270_x6uYqmx31kE_00001.wav G5_VER/id10300_ize_eiCFEg0_00003.wav\n',
 '1 G5_VER/id10270_x6uYqmx31kE_00001.wav G5_VER/id10270_GWXujl-xAVM_00017.wav\n',
 '0 G5_VER/id10270_x6uYqmx31kE_00001.wav G5_VER/id10273_0OCW1HUxZyg_00001.wav\n',
 '1 G5_VER/id10270_x6uYqmx31kE_00001.wav G5_VER/id10270_8jEAjG6SegY_00022.wav\n',
 '0 G5_VER/id10270_x6uYqmx31kE_00001.wav G5_VER/id10284_Uzxv7Axh3Z8_00001.wav\n',
 '1 G5_VER/id10270_x6uYqmx31kE_00001.wav G5_VER/id10270_GWXujl-xAVM_00033.wav\n',
 '0 G5_VER/id10270_x6uYqmx31kE_00001.wav G5_VER/id10284_7yx9A0yzLYk_00029.wav\n',
 '1 G5_VER/id10270_x6uYqmx31kE_00002.wav G5_VER/id10270_5r0dWxy17C8_00026.wav\n',
 '0 G5_VER/id10270_x6uYqmx31kE_00002.wav G5_VER/id10285_m-uILToQ9ss_00009.wav\n',
 '1 G5_VER/id10270_x6uYqmx31kE_00002.wav G5_VER/id10270_GWXujl-xAVM_00035.wav\n',
 '0 G5_VER/id10270_x6uYqmx31kE_00002.wav G5_VER/id10306_uzt36PBzT2w_00001.wav\n',
 '1 G5_VER/id102

In [72]:
# write ver_new to .txt file
with open('/nvme/zhiyong/fllearn_data/ver_list_G5_VER.txt', 'w') as f:
    for i in ver_new:
        f.write(i)

# G6 TEST VER

In [89]:
import glob
a = glob.glob('/nvme/zhiyong/voxceleb/vox1/test/wav/*')

In [90]:
b = a

In [91]:
b_list = []
for i in b:
    b_list.extend(glob.glob(i+'/*'))

In [92]:
b_list

['/nvme/zhiyong/voxceleb/vox1/test/wav/id10301/AeRSD9jIdSg',
 '/nvme/zhiyong/voxceleb/vox1/test/wav/id10301/rXRbmL7nzIo',
 '/nvme/zhiyong/voxceleb/vox1/test/wav/id10301/6uqGkiR2oTI',
 '/nvme/zhiyong/voxceleb/vox1/test/wav/id10301/UkrkPwZd77Y',
 '/nvme/zhiyong/voxceleb/vox1/test/wav/id10301/BVFTRGHuk3M',
 '/nvme/zhiyong/voxceleb/vox1/test/wav/id10301/Eu8Bj1gx5_4',
 '/nvme/zhiyong/voxceleb/vox1/test/wav/id10301/vhzF7dhGJzI',
 '/nvme/zhiyong/voxceleb/vox1/test/wav/id10301/eVfmXU0RtRo',
 '/nvme/zhiyong/voxceleb/vox1/test/wav/id10301/VIAnMw7J5tI',
 '/nvme/zhiyong/voxceleb/vox1/test/wav/id10301/hX6LjjVtnGs',
 '/nvme/zhiyong/voxceleb/vox1/test/wav/id10274/wH6ebv-K8AA',
 '/nvme/zhiyong/voxceleb/vox1/test/wav/id10274/YCIIiIR9B60',
 '/nvme/zhiyong/voxceleb/vox1/test/wav/id10274/7rkVfW2yw0c',
 '/nvme/zhiyong/voxceleb/vox1/test/wav/id10274/xbGp2KSUr0E',
 '/nvme/zhiyong/voxceleb/vox1/test/wav/id10274/89UnsytX6Y8',
 '/nvme/zhiyong/voxceleb/vox1/test/wav/id10274/L9q6XabEpEI',
 '/nvme/zhiyong/voxceleb

In [93]:
def generate_wave(path, out_path):
    fft_len = 512
    Lg_t = 0.100                # filter size in seconds
    Lg = np.ceil(Lg_t*16000)       # in samples
    absor, size_coef = room_properties('medium', 'medium')
    room = pra.ShoeBox(
        p=np.array([4*size_coef, 3*size_coef, 2.5]),
        materials=pra.Material(absor),
        fs=16000,
        max_order=3,
        ray_tracing=True,
        air_absorption=True,
    )
    room.set_ray_tracing(receiver_radius=0.1, n_rays=10000, energy_thres=1e-5)

    # specify signal source
    fs, signal = wavfile.read(path)

    # add source to 2D room
    room.add_source([1.,1.,1.], signal=signal)

    # add noise source
    fs, noise = wavfile.read("/nvme/zhiyong/musan/noise/free-sound/noise-free-sound-0000.wav")

    room.add_source([8.,7.,1.], signal=noise[:len(signal)], delay=0.)

    # add microphone array
    R = pra.circular_2D_array(center=[8.,2.], M=6, phi0=0, radius=0.05)
    R = np.concatenate((R, np.ones((1, 6))))

    mics = pra.Beamformer(R, fs, N=fft_len, Lg=Lg)
    room.add_microphone_array(mics)

    # Compute DAS weights
    mics.rake_delay_and_sum_weights(room.sources[0][:1])

    room.simulate()
    signal_das = mics.process(FD=False)
    to_wav(out_path, fs, signal_das, norm=True, bitdepth=np.int16)

In [94]:
glob.glob('/nvme/zhiyong/voxceleb/vox1/test/wav/id10301/AeRSD9jIdSg/*')

['/nvme/zhiyong/voxceleb/vox1/test/wav/id10301/AeRSD9jIdSg/00010.wav',
 '/nvme/zhiyong/voxceleb/vox1/test/wav/id10301/AeRSD9jIdSg/00004.wav',
 '/nvme/zhiyong/voxceleb/vox1/test/wav/id10301/AeRSD9jIdSg/00001.wav',
 '/nvme/zhiyong/voxceleb/vox1/test/wav/id10301/AeRSD9jIdSg/00006.wav',
 '/nvme/zhiyong/voxceleb/vox1/test/wav/id10301/AeRSD9jIdSg/00005.wav',
 '/nvme/zhiyong/voxceleb/vox1/test/wav/id10301/AeRSD9jIdSg/00009.wav',
 '/nvme/zhiyong/voxceleb/vox1/test/wav/id10301/AeRSD9jIdSg/00007.wav',
 '/nvme/zhiyong/voxceleb/vox1/test/wav/id10301/AeRSD9jIdSg/00008.wav',
 '/nvme/zhiyong/voxceleb/vox1/test/wav/id10301/AeRSD9jIdSg/00003.wav',
 '/nvme/zhiyong/voxceleb/vox1/test/wav/id10301/AeRSD9jIdSg/00002.wav']

In [95]:
for i in b_list:
    c = glob.glob(i+'/*')
    for j in c:
        label = j.split('/')[-3]
        out_path = '/nvme/zhiyong/fllearn_data/G6_VER/'
        generate_wave(j, out_path+label+'_'+j.split('/')[-2]+'_'+j.split('/')[-1])

/tmp/ipykernel_43156/3403348603.py:50: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  float_types = [float, np.float, np.float32, np.float64]


In [73]:
ver_ori_path = '/nvme/zhiyong/sdsv21/vox_o_triallist.txt'

In [74]:
with open(ver_ori_path, 'r') as f:
    ver_ori = f.readlines()

In [75]:
ver_ori

['1 vox1/test/wav/id10270/x6uYqmx31kE/00001.wav vox1/test/wav/id10270/8jEAjG6SegY/00008.wav\n',
 '0 vox1/test/wav/id10270/x6uYqmx31kE/00001.wav vox1/test/wav/id10300/ize_eiCFEg0/00003.wav\n',
 '1 vox1/test/wav/id10270/x6uYqmx31kE/00001.wav vox1/test/wav/id10270/GWXujl-xAVM/00017.wav\n',
 '0 vox1/test/wav/id10270/x6uYqmx31kE/00001.wav vox1/test/wav/id10273/0OCW1HUxZyg/00001.wav\n',
 '1 vox1/test/wav/id10270/x6uYqmx31kE/00001.wav vox1/test/wav/id10270/8jEAjG6SegY/00022.wav\n',
 '0 vox1/test/wav/id10270/x6uYqmx31kE/00001.wav vox1/test/wav/id10284/Uzxv7Axh3Z8/00001.wav\n',
 '1 vox1/test/wav/id10270/x6uYqmx31kE/00001.wav vox1/test/wav/id10270/GWXujl-xAVM/00033.wav\n',
 '0 vox1/test/wav/id10270/x6uYqmx31kE/00001.wav vox1/test/wav/id10284/7yx9A0yzLYk/00029.wav\n',
 '1 vox1/test/wav/id10270/x6uYqmx31kE/00002.wav vox1/test/wav/id10270/5r0dWxy17C8/00026.wav\n',
 '0 vox1/test/wav/id10270/x6uYqmx31kE/00002.wav vox1/test/wav/id10285/m-uILToQ9ss/00009.wav\n',
 '1 vox1/test/wav/id10270/x6uYqmx31kE/00

In [76]:
ver_new = []
for i in ver_ori:
    i = i.strip()
    ver_new.append(i.split(' ')[0]+' '
                'G6_VER/'+i.split(' ')[1].split('/')[-3]+'_'+i.split(' ')[1].split('/')[-2]+'_'+i.split(' ')[1].split('/')[-1]+' '
                'G6_VER/'+i.split(' ')[2].split('/')[-3]+'_'+i.split(' ')[2].split('/')[-2]+'_'+i.split(' ')[2].split('/')[-1]+'\n')

In [77]:
ver_new

['1 G6_VER/id10270_x6uYqmx31kE_00001.wav G6_VER/id10270_8jEAjG6SegY_00008.wav\n',
 '0 G6_VER/id10270_x6uYqmx31kE_00001.wav G6_VER/id10300_ize_eiCFEg0_00003.wav\n',
 '1 G6_VER/id10270_x6uYqmx31kE_00001.wav G6_VER/id10270_GWXujl-xAVM_00017.wav\n',
 '0 G6_VER/id10270_x6uYqmx31kE_00001.wav G6_VER/id10273_0OCW1HUxZyg_00001.wav\n',
 '1 G6_VER/id10270_x6uYqmx31kE_00001.wav G6_VER/id10270_8jEAjG6SegY_00022.wav\n',
 '0 G6_VER/id10270_x6uYqmx31kE_00001.wav G6_VER/id10284_Uzxv7Axh3Z8_00001.wav\n',
 '1 G6_VER/id10270_x6uYqmx31kE_00001.wav G6_VER/id10270_GWXujl-xAVM_00033.wav\n',
 '0 G6_VER/id10270_x6uYqmx31kE_00001.wav G6_VER/id10284_7yx9A0yzLYk_00029.wav\n',
 '1 G6_VER/id10270_x6uYqmx31kE_00002.wav G6_VER/id10270_5r0dWxy17C8_00026.wav\n',
 '0 G6_VER/id10270_x6uYqmx31kE_00002.wav G6_VER/id10285_m-uILToQ9ss_00009.wav\n',
 '1 G6_VER/id10270_x6uYqmx31kE_00002.wav G6_VER/id10270_GWXujl-xAVM_00035.wav\n',
 '0 G6_VER/id10270_x6uYqmx31kE_00002.wav G6_VER/id10306_uzt36PBzT2w_00001.wav\n',
 '1 G6_VER/id102

In [42]:
# write ver_new to .txt file
with open('/nvme/zhiyong/fllearn_data/ver_list_G6_VER.txt', 'w') as f:
    for i in ver_new:
        f.write(i)